In [112]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.font_manager import FontProperties

# matplotlib.style.use('ggplot')
import matplotlib
# matplotlib.style.use('ggplot')
#matplotlib.style.use('default')
from numpy import *
import numpy as np
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (8, 6)
import subprocess,shlex,sys,select,re,StringIO
def readHive(sql):
    text = ''
    err = ''
    cmd = shlex.split('hive -e '+ '"set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;'+sql+'"')
    print >>sys.stderr, cmd
    sys.stderr.flush()
    proc = subprocess.Popen(cmd, bufsize=0, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    while proc.poll() is None:
        ins, outs, errs = select.select([proc.stdout, proc.stderr], [], [])
        if proc.stdout in ins:
            text += proc.stdout.read()
        if proc.stderr in ins:
            c = proc.stderr.read(1)
            if c == '\n':
                print >>sys.stderr, err
                sys.stderr.flush()
                err = ''
            else:
                err += c
    text += proc.stdout.read()
    print >>sys.stderr, proc.stderr.read()
    sys.stderr.flush()
    header, content = text.split('\n',1)
    header = re.sub('\w*\.','', header)
    return pd.read_table(StringIO.StringIO(header+'\n'+content), error_bad_lines = False)

def f(x):
    x=str(x)
    tmp=x.split("//")
    if len(tmp)<=1:
        return x
    return '.'.join(x.strip().split('//')[1].split('/')[0].split('.')[-3:])

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [72]:
clk=readHive("select * from fraud_free.dclkv2 where year=2016 and month=10 and day=12")
ld=readHive("select * from etl_output.landing where year=2016 and month=10 and day=12")

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select * from fraud_free.dclkv2 where year=2016 and month=10 and day=12']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
OK
s, Fetched: 619125 row(s)

/home/jay/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select * from etl_output.landing where year=2016 and month=10 and day=12']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
OK
Fetched: 111002 row(s)



In [113]:
clk['domain']=clk['url'].apply(f)

In [114]:
j=pd.merge(left=clk,right=ld,how='left',on='request_id')

In [115]:
j.query('fraud_flag!=fraud_flag & company_id==630')['request_id'].ucount()

AttributeError: 'Series' object has no attribute 'ucount'

In [122]:
clk.query('fraud_flag==fraud_flag & domain=="dianying.2345.com"')['request_id'].count()

16090

In [58]:
j.dtypes

zid_x               object
mark               float64
timestamp_x          int64
url                 object
refer_x             object
target_url          object
ip_x                 int64
ua_x                object
request_id          object
vendor_id            int64
project_id           int64
company_id           int64
campaign_id          int64
adgroup_id           int64
ad_id                int64
material_id          int64
material_type       object
adslot_id           object
rtb_time             int64
pid                 object
template_id          int64
secret_key          object
uuid_type            int64
uuid               float64
fraud_flag         float64
device_id           object
model_type          object
bid_model          float64
wrap_func           object
segment_type       float64
pctr               float64
adj_pctr           float64
spam_ids            object
minute_x             int64
hour_x               int64
day_x                int64
month_x              int64
y

In [3]:
ip=readHive("select vendor_id,count(distinct hour) from archive.rtbv2 where year=2016 and month=10 and day=7 group by vendor_id,ip")

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select vendor_id,count(1) from archive.rtbv2 where year=2016 and month=10 and day=7 group by vendor_id,ip']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
Query ID = changbin_20161008192356_8ea32016-6060-4883-b762-412b6ea589d3
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1009
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1471856463785_1054957, Tracking URL = http://TS-NN-001:50030/proxy/application_1471856463785_1054957/
Kill Command = /var/bh/hadoop/bin/hadoop job  -kill job_1471856463785_1054957
Hadoop job information f

In [5]:
ip

,vendor_id,_c1
0,1,648
1,1,853
2,1,292
3,1,274
4,1,826
5,1,639
6,1,6337
7,1,1626
8,1,129
9,1,420


In [11]:
ip['_c1'].describe()

count    3.888510e+08
mean     4.253985e+01
std      2.606411e+03
min      1.000000e+00
25%      2.000000e+00
50%      7.000000e+00
75%      2.300000e+01
max      3.844252e+07
Name: _c1, dtype: float64

In [16]:
user['_c1'].quantile([.80,.90,.95,.99])

0.80     11.0
0.90     24.0
0.95     45.0
0.99    150.0
Name: _c1, dtype: float64

In [23]:
user=readHive("select vendor_id, count(1) from archive.rtbv2 where year=2016 and month=10 and day=7 group by vendor_id,pid")

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select vendor_id, count(1) from archive.rtbv2 where year=2016 and month=10 and day=7 group by vendor_id,pid']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties


KeyboardInterrupt: 

In [ ]:
useractive=readHive("select vendor_id, count(distinct hour) from archive.rtbv2 where year=2016 and month=10 and day=7 group by vendor_id,pid")

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select vendor_id, count(distinct hour) from archive.rtbv2 where year=2016 and month=10 and day=7 group by vendor_id,pid']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
Query ID = changbin_20161010140318_dfad7a22-0f37-44bc-bffe-e3d329c1c7c0
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1009
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1471856463785_1099601, Tracking URL = http://TS-NN-001:50030/proxy/application_1471856463785_1099601/
Kill Command = /var/bh/hadoop/bin/hadoop job  -kill job_1471856463785_1099601
Hadoop job

In [5]:
useractive

,vendor_id,c1
0,0,17
1,1,2
2,1,2
3,1,6
4,1,2
5,1,1
6,1,1
7,1,3
8,1,4
9,1,1


In [1]:
for i in [1,2,5,9,15,21]:
    print "-----------"+str(i)
    print useractive.query("vendor_id=="+str(i))['c1'].describe()
    print useractive.query("vendor_id=="+str(i))['c1'].quantile([.80,.90,.95,.99,.999])

-----------1


NameError: name 'useractive' is not defined